In [362]:
import pandas as pd
import numpy as np

In [363]:
# read our data from the 'K-12 SSDB (Public).xlsx' spreadsheet
df = pd.read_excel('K-12 SSDB (Public).xlsx')

In [364]:
# change column header to correct information
df.columns = df.iloc[0]

In [365]:
# remove the row that is now our column
df = df.drop([0])

In [366]:
# sort our data by most recent shooting
df = df.sort_index(axis=0, ascending=False)

In [367]:
# order the index now from 0 on
df = df.reset_index(drop=True)

In [368]:
df.head()

,Date,School,City,State,Reliability Score (1-5),Killed (includes shooter),Wounded,Total Injured/Killed Victims,Gender of Victims (M/F/Both),Victim's Affiliation w/ School,...,Caliber of Firearm,Number of Shooters,Shooter Name,Shooter Age,Shooter Gender,Shooter Race,Shooter Ethnicity,Shooter's Affiliation with School,Shooter had an accomplice who did not fire gun (Y/N),Hostages Taken (Y/N)
0,2019-02-17 00:00:00,Eaglecrest High School,Aurora,CO,4,1,0,1,Male,Principal/Vice-Principal,...,Unknown,1,Marcus Johnson,31,M,Black,Not Hispanic or Latino,No Relation,N,N
1,2019-02-14 00:00:00,V. Sue Cleveland High School,Rio Rancho,NM,4,0,0,0,No Victims,No Victims,...,Unknown,1,Joshua Owen,16,M,Unknown,Unknown,Student,N,N
2,2019-02-08 00:00:00,Frederick Douglass High School,Baltimore,MD,4,0,1,1,Male,Other Staff,...,Unknown,1,Neil Davis,25,M,Black,Not Hispanic or Latino,No Relation,N,N
3,2019-02-05 00:00:00,Minneapolis School District Bus,Minneapolis,MN,3,0,1,1,Male,Other Staff,...,Unknown,1,Kenneth Lilli,31,M,Unknown,Unknown,No Relation,N,N
4,2019-01-31 00:00:00,Atascocita High School,Atascocita,TX,4,0,1,1,Male,Student,...,Unknown,1,Unknown,NaN,M,Unknown,Unknown,Student,Y,N


In [369]:
# Change timestap to 'YYYY-MM-DD'
df['Date'] = df.Date.map(lambda x: x.strftime('%Y-%m-%d'))

In [370]:
# Create a column holding just the year of occurence ('YYYY')
df['Year'] = [date[0:4] for date in df['Date']]

In [371]:
# read our data from the 'Gun_laws.xlsx' spreadsheet
gun_laws_df = pd.read_excel('Gun_laws.xlsx')

In [372]:
gun_laws_df.head()

,State,2012,2013,2014,2015,2016,2017,2018
0,AL,D,D-,F,F,F,F,F
1,AK,F,F,F,F,F,F,F
2,AZ,F,F,F,F,F,F,F
3,AR,F,F,F,F,F,F,F
4,CA,A-,A-,A-,A-,A,A,A


In [373]:
gun_laws_df = gun_laws_df.set_index('State')

In [374]:
df.head()

,Date,School,City,State,Reliability Score (1-5),Killed (includes shooter),Wounded,Total Injured/Killed Victims,Gender of Victims (M/F/Both),Victim's Affiliation w/ School,...,Number of Shooters,Shooter Name,Shooter Age,Shooter Gender,Shooter Race,Shooter Ethnicity,Shooter's Affiliation with School,Shooter had an accomplice who did not fire gun (Y/N),Hostages Taken (Y/N),Year
0,2019-02-17,Eaglecrest High School,Aurora,CO,4,1,0,1,Male,Principal/Vice-Principal,...,1,Marcus Johnson,31,M,Black,Not Hispanic or Latino,No Relation,N,N,2019
1,2019-02-14,V. Sue Cleveland High School,Rio Rancho,NM,4,0,0,0,No Victims,No Victims,...,1,Joshua Owen,16,M,Unknown,Unknown,Student,N,N,2019
2,2019-02-08,Frederick Douglass High School,Baltimore,MD,4,0,1,1,Male,Other Staff,...,1,Neil Davis,25,M,Black,Not Hispanic or Latino,No Relation,N,N,2019
3,2019-02-05,Minneapolis School District Bus,Minneapolis,MN,3,0,1,1,Male,Other Staff,...,1,Kenneth Lilli,31,M,Unknown,Unknown,No Relation,N,N,2019
4,2019-01-31,Atascocita High School,Atascocita,TX,4,0,1,1,Male,Student,...,1,Unknown,NaN,M,Unknown,Unknown,Student,Y,N,2019


In [376]:
gun_laws_df.head()

,2012,2013,2014,2015,2016,2017,2018
State,,,,,,,
AL,D,D-,F,F,F,F,F
AK,F,F,F,F,F,F,F
AZ,F,F,F,F,F,F,F
AR,F,F,F,F,F,F,F
CA,A-,A-,A-,A-,A,A,A


In [377]:
df['Year'] = pd.to_numeric(df['Year'])

In [378]:
gun_laws_df[gun_laws_df.columns[gun_laws_df.columns == 2018]].loc['AL'].values[0]

'F'

In [400]:
df['Grade'] = 'NaN'
grades = []
for idx, row in df.iterrows():
    try:
        grade = gun_laws_df[gun_laws_df.columns[gun_laws_df.columns == row['Year']]].loc[row['State']].values[0]
        grades.append(grade)  
    except:
        pass
print(grades)
# df[df['Year'] == 2013]
# df[df['Grade'] == 'A']


['F', 'D-', 'F', 'C+', 'F', 'B+', 'C', 'D', 'A', 'F', 'F', 'D', 'C', 'F', 'D-', 'D', 'D-', 'C+', 'F', 'F', 'F', 'A-', 'C+', 'D', 'A', 'F', 'B+', 'C-', 'D', 'D-', 'B+', 'A', 'B+', 'A-', 'C', 'C', 'D', 'B', 'C', 'B+', 'C-', 'C-', 'A-', 'B+', 'F', 'D', 'F', 'D-', 'F', 'F', 'F', 'C+', 'D', 'F', 'D-', 'F', 'F', 'F', 'F', 'B+', 'A', 'F', 'D-', 'F', 'C-', 'C', 'F', 'A-', 'F', 'A-', 'D', 'F', 'D', 'A', 'F', 'F', 'F', 'F', 'B+', 'D', 'C-', 'C-', 'D-', 'A-', 'A-', 'C+', 'A', 'C+', 'C', 'F', 'F', 'F', 'F', 'D+', 'F', 'C', 'B+', 'B', 'F', 'A', 'C', 'F', 'F', 'D-', 'F', 'D', 'D', 'A', 'C', 'D-', 'F', 'A', 'F', 'F', 'C', 'C', 'A', 'F', 'B+', 'B', 'D', 'F', 'F', 'F', 'C-', 'F', 'B+', 'F', 'D', 'F', 'F', 'F', 'A', 'C', 'A', 'C+', 'F', 'F', 'D', 'F', 'F', 'F', 'C-', 'F', 'C+', 'C', 'C', 'C', 'F', 'A', 'F', 'D', 'F', 'F', 'B+', 'F', 'F', 'C', 'F', 'C', 'F', 'D', 'F', 'A-', 'B+', 'A-', 'F', 'F', 'F', 'F', 'A-', 'C', 'F', 'C-', 'F', 'D', 'F', 'C', 'F', 'F', 'C', 'C', 'D-', 'D-', 'F', 'B', 'F', 'F', 'F', '

In [ ]:
# for year in df['Year']: 
#     for state in df['State']:
        
#         print(year, state)

In [ ]:
# loop through big data
# get year n code
# use laws code to get grade based on ye